In [70]:
!pip install torchmetrics

In [71]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [220]:
# Load Data
df_raw=pd.read_csv('../content/water_potability.csv')
#df_raw=pd.read_csv('water_potability.csv')

# drop Nan data and reset index
df_dropped = df_raw.dropna()
df = df_dropped.reset_index(drop=True)
print("raw data has", len(df_raw), " after drop has",  len(df))

raw data has 3276  after drop has 2011


In [221]:
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
1,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
2,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
3,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
4,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0


In [222]:
# Convert from Pandas dataframe to Numpy array
ndf = df.to_numpy()

In [223]:
print("Data size after converting to numpy is", len(ndf))

Data size after converting to numpy is 2011


In [224]:
# split into input (X) and output (y) variables
X = ndf[:,0:9]
y = ndf[:,9]

# create a tensor out of NumPy arrays
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

In [233]:
# define model: 3 layers

model = nn.Sequential(
    nn.Linear(9, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
).to(device)


# define model: 4 layers
'''
model = nn.Sequential(
    nn.Linear(9, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
).to(device)
'''

# define model: 5 layers
'''
model = nn.Sequential(
    nn.Linear(9, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
).to(device)
'''

# define model: 6 layers
'''
model = nn.Sequential(
    nn.Linear(9, 16),
    nn.ReLU(),
    nn.Linear(16, 32),
    nn.ReLU(),
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
).to(device)
'''

'\nmodel = nn.Sequential(\n    nn.Linear(9, 16),\n    nn.ReLU(),\n    nn.Linear(16, 32),\n    nn.ReLU(),\n    nn.Linear(32, 32),\n    nn.ReLU(),\n    nn.Linear(32, 16),\n    nn.ReLU(),\n    nn.Linear(16, 1)\n).to(device)\n'

In [234]:
print(model)

Sequential(
  (0): Linear(in_features=9, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=1, bias=True)
)


In [235]:
# define loss function and optimizer with default learning rate as 0.001
loss_fn = nn.BCEWithLogitsLoss().to(device)  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [236]:
# Set manual seed since nn.Parameter are randomly initialzied
torch.manual_seed(42)

precision_metric = BinaryPrecision().to(device)
accuracy_metric = BinaryAccuracy().to(device)
recall_metric = BinaryRecall().to(device)

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []
train_accuray_values = []
test_accuray_values = []
test_precision_values = []
test_recall_values = []


In [241]:

# Train the model
n_epochs = 500

# split dataset into the batch of 10
batch_size = 128

# run as one batch on entire data set
#batch_size = len(X_train)

for epoch in range(n_epochs):
    indices = torch.randperm(len(X_train))
    for i in range(0, len(X_train), batch_size):
        model.train()

        batch_indices = indices[i:i+batch_size]
        Xbatch = X_train[batch_indices]
        ybatch = y_train[batch_indices]

        Xbatch = Xbatch.to(device)
        ybatch = ybatch.to(device)

        # Forward pass (model outputs raw logits)
        ybatch_logits = model(Xbatch)
        ybatch_pred = torch.round(torch.sigmoid(ybatch_logits))

        # Calculate the loss
        loss = loss_fn(ybatch_logits, ybatch)
        train_accuracy = accuracy_metric(ybatch_pred.to(device), ybatch.to(device))

        #Zero gradients
        optimizer.zero_grad()

        # Perform backpropagation on the loss
        loss.backward()

        # Step the optimizer (gradient descent)
        optimizer.step()

         # print loss
        #print(f'Finished epoch {epoch}, latest loss {loss}')

    ### Testing

    # Put the model in evaluation mode
    model.eval()

    with torch.inference_mode():
        # 1. Forward pass on test data
        test_logits = model(X_test.to(device))
        test_pred = torch.round(torch.sigmoid(test_logits))

        # 2. Caculate loss on test data
        test_loss = loss_fn(test_logits, y_test)
        test_accuracy = accuracy_metric(test_pred, y_test)

        test_precision = precision_metric(test_pred, y_test)
        test_recall = recall_metric(test_pred, y_test)

      # Print out what's happening
        epoch_count.append(epoch)
        train_loss_values.append(loss.detach().cpu().numpy())
        test_loss_values.append(test_loss.detach().cpu().numpy())
        train_accuray_values.append(train_accuracy.detach().cpu().numpy())
        test_accuray_values.append(test_accuracy.detach().cpu().numpy())
        test_precision_values.append(test_precision.detach().cpu().numpy())
        test_recall_values.append(test_recall.detach().cpu().numpy())




In [242]:
# Predict: Turn model into evaluation mode
model.eval()

# Make predictions on the test data
with torch.inference_mode():
    y_preds_test = model(X_test)
    y_preds_train = model(X_train)
y_preds_test
y_preds_train

train_accuracy = accuracy_metric(y_preds_train, y_train)
test_accuracy = accuracy_metric(y_preds_test, y_test)
preds_precision = precision_metric(y_preds_test, y_test)
preds_recall = recall_metric(y_preds_test, y_test)
print('Accuracy of the network on train data: {:.5f}'.format(train_accuracy))
print('Accuracy of the network on test data: {:.5f}'.format(test_accuracy))
print('Precision of the network on test data: {:.5f}'.format(preds_precision))
print('Recall of the network on test data: {:.5f}'.format(preds_recall))

Accuracy of the network on train data: 0.58067
Accuracy of the network on test data: 0.55795
Precision of the network on test data: 0.45679
Recall of the network on test data: 0.29249
